<h1>Cooking time Prediction from cooking directions

In [ ]:
%pip install mlflow

Python interpreter will be restarted.
Collecting mlflow
 Downloading mlflow-1.30.0-py3-none-any.whl (17.0 MB)
Requirement already satisfied: click<9,>=7.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (8.1.3)
Collecting alembic<2
 Downloading alembic-1.10.2-py3-none-any.whl (212 kB)
Requirement already satisfied: pandas<2 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.0.1)
Collecting importlib-metadata!=4.7.0,<6,>=3.7.0
 Downloading importlib_metadata-5.2.0-py3-none-any.whl (21 kB)
Collecting querystring-parser<2
 Downloading querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
Requirement already satisfied: protobuf<5,>=3.12.0 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (3.20.1)
Collecting sqlparse<1,>=0.4.0
 Downloading sqlparse-0.4.3-py3-none-any.whl (42 kB)
Requirement already satisfied: numpy<2 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.18.1)
Collecting sqlalchemy<2,>=1.4.0
 Downloading SQLAlchemy-1.4.46-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
Collecting docker<7,>=4.0.0
 Downloading docker-6.0.1-py3-none-any.whl (147 kB)
Requirement already satisfied: requests<3,>=2.17.3 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2.22.0)
Collecting gitpython<4,>=2.1.0
 Downloading GitPython-3.1.31-py3-none-any.whl (184 kB)
Collecting gunicorn<21; platform_system != "Windows"
 Downloading gunicorn-20.1.0-py3-none-any.whl (79 kB)
Requirement already satisfied: scipy<2 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (1.4.1)
Collecting cloudpickle<3
 Downloading cloudpickle-2.2.1-py3-none-any.whl (25 kB)
Collecting packaging<22
 Downloading packaging-21.3-py3-none-any.whl (40 kB)
Requirement already satisfied: entrypoints<1 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (0.3)
Collecting databricks-cli<1,>=0.8.7
 Downloading databricks-cli-0.17.5.tar.gz (82 kB)
Requirement already satisfied: pytz<2023 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (2019.3)
Collecting Flask<3
 Downloading Flask-2.2.3-py3-none-any.whl (101 kB)
Requirement already satisfied: pyyaml<7,>=5.1 in /databricks/python3/lib/python3.7/site-packages (from mlflow) (6.0)
Collecting prometheus-flask-exporter<1
 Downloading prometheus_flask_exporter-0.22.3-py3-none-any.whl (18 kB)
Requirement already satisfied: typing-extensions>=4 in /databricks/python3/lib/python3.7/site-packages (from alembic<2->mlflow) (4.5.0)
Collecting Mako
 Downloading Mako-1.2.4-py3-none-any.whl (78 kB)
Collecting importlib-resources; python_version < "3.9"
 Downloading importlib_resources-5.12.0-py3-none-any.whl (36 kB)
Requirement already satisfied: python-dateutil>=2.6.1 in /databricks/python3/lib/python3.7/site-packages (from pandas<2->mlflow) (2.8.1)
Requirement already satisfied: zipp>=0.5 in /databricks/python3/lib/python3.7/site-packages (from importlib-metadata!=4.7.0,<6,>=3.7.0->mlflow) (3.15.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from querystring-parser<2->mlflow) (1.14.0)
Collecting greenlet!=0.4.17; python_version >= "3" and (platform_machine == "aarch64" or (platform_machine == "ppc64le" or (platform_machine == "x86_64" or (platform_machine == "amd64" or (platform_machine == "AMD64" or (platform_machine == "win32" or platform_machine == "WIN32"))))))
 Downloading greenlet-2.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (566 kB)
Collecting websocket-client>=0.32.0
 Downloading websocket_client-1.5.1-py3-none-any.whl (55 kB)
Collecting urllib3>=1.26.0
 Downloading urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
Requirement already satisfied: idna<2.9,>=2.5 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.17.3->mlflow) (2.8)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.7/site-packages (from requests<3,>=2.17.3->mlflow) (2020.6.20)
Requirem

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext,Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressionModel

In [ ]:
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt

In [ ]:
sc._conf.get('spark.executor.memory'),sc._conf.get('spark.driver.memory')

Out[3]: ('44632m', None)

In [ ]:
sc._conf.set('spark.executor.memory','64g'),sc._conf.set('spark.driver.memory','64g')

Out[4]: (<pyspark.conf.SparkConf at 0x7f3536a2e750>,
 <pyspark.conf.SparkConf at 0x7f3536a2e750>)

In [ ]:
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 spark://10.169.221.216:7077 
 AppName 
 Databricks Shell

<h4>Reading the data from MongoDB Atlas

In [ ]:
df = spark.read.format("mongo").option('uri',f'mongodb+srv://{cluster_detail}/project_db.recipes').load()
df.count()

Out[6]: 21885

In [ ]:
df.printSchema() #Getting the schema

root
-- _id: string (nullable = true)
-- actual_review_cnt: integer (nullable = true)
-- details: struct (nullable = true)
 |-- Active Time: string (nullable = true)
 |-- Additional Time: string (nullable = true)
 |-- Bake Time: string (nullable = true)
 |-- Chill Time: string (nullable = true)
 |-- Cook Time: string (nullable = true)
 |-- Cool Time: string (nullable = true)
 |-- Freeze Time: string (nullable = true)
 |-- Marinate Time: string (nullable = true)
 |-- Prep Time: string (nullable = true)
 |-- Servings: string (nullable = true)
 |-- Stand Time: string (nullable = true)
 |-- Total Time: string (nullable = true)
 |-- Yield: string (nullable = true)
-- directions: array (nullable = true)
 |-- element: string (containsNull = true)
-- hierarchy: array (nullable = true)
 |-- element: string (containsNull = true)
-- ingredients: array (nullable = true)
 |-- element: array (containsNull = true)
 | |-- element: string (containsNull = true)
-- intro: string (nullable = true)
-- name: string (nullable = true)
-- nutrition: struct (nullable = true)
 |-- Calcium: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Calories: string (nullable = true)
 |-- Cholesterol: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Dietary Fiber: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Iron: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Potassium: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Protein: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Saturated Fat: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Servings: string (nullable = true)
 |-- Sodium: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Total Carbohydrate: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Total Fat: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Total Sugars: array (nullable = true)
 | |-- element: string (containsNull = true)
 |-- Vitamin C: array (nullable = true)
 | |-- element: string (containsNull = true)
-- picture_cnt: string (nullable = true)
-- publish_date: string (nullable = true)
-- publisher_name: string (nullable = true)
-- publisher_url: string (nullable = true)
-- rating: string (nullable = true)
-- rating_cnt: string (nullable = true)
-- review_cnt: integer (nullable = true)
-- reviews: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- created: string (nullable = true)
 | |-- displayName: string (nullable = true)
 | |-- docId: integer (nullable = true)
 | |-- helpfulCount: integer (nullable = true)
 | |-- legacyFeedbackId: string (nullable = true)
 | |-- madeIt: integer (nullable = true)
 | |-- photos: array (nullable = true)
 | | |-- element: struct (containsNull = true)
 | | | |-- url: string (nullable = true)
 | | | |-- mimeType: string (nullable = true)
 | | | |-- width: integer (nullable = true)
 | | | |-- height: integer (nullable = true)
 | |-- profileUrl: string (nullable = true)
 | |-- review: string (nullable = true)
 | |-- starRating: integer (nullable = true)
 | |-- userId: string (nullable = true)
-- similar_recipes: array (nullable = true)
 |-- element: string (containsNull = true)
-- text: string (nullable = true)
-- url: string (nullable = true)

In [ ]:
df.select('directions').show(1,False) # We have to embedd this directions string

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
directions |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[Half-fill a highball glass with ice cubes. Measure in the coffee liqueur and vodka. Pour cola to within 1/2 inch of the top. finish with enough Irish stout to form a 'head' on the top. Stir and serve.]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row

<h4>De-array the directions and using regex to remove punctuations and keeping the fractions like 1/2 and 1/4

In [ ]:
#We need the numbers and fractions as they can indicate the amount of time taken for a recipe
df1 = df.withColumn('directions', explode('directions'))
df_time = df1.select('name','details.Total Time',regexp_replace("directions", "[^\w\s/]|(?<!\d)\/(?!\d)", "").alias('cooking_steps'))
df_time.select('cooking_steps').show(3,False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
cooking_steps |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Halffill a highball glass with ice cubes Measure in the coffee liqueur and vodka Pour cola to within 1/2 inch of the top finish with enough Irish stout to form a head on the top Stir and serve|
Preheat oven to 375 degrees F 190 degrees C Press pie crust into a pie plate folding crust over the edge and crimping into place |
Blend sugar squash milk eggs cinnamon and salt together in a blender or with a whisk until smooth Pour mixture into the prepared pie crust |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 3 rows

<h4>Using Bert's Sentence Transformer based on MiniLM to embedd our direction string

In [ ]:
emb_model = SentenceTransformer('all-MiniLM-L6-v2')

# udf for getting embeddings
def get_embeddings(str):
    return emb_model.encode(str, show_progress_bar=True).tolist()

# get the embeddings for the ingredients_str column
get_embeddings_udf = udf(get_embeddings, ArrayType(FloatType()))

Downloading (…)e9125/.gitattributes: 0%| | 0.00/1.18k [00:00<?, ?B/s]Downloading (…)e9125/.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 137kB/s]
Downloading (…)_Pooling/config.json: 0%| | 0.00/190 [00:00<?, ?B/s]Downloading (…)_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 22.6kB/s]
Downloading (…)7e55de9125/README.md: 0%| | 0.00/10.6k [00:00<?, ?B/s]Downloading (…)7e55de9125/README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 3.21MB/s]
Downloading (…)55de9125/config.json: 0%| | 0.00/612 [00:00<?, ?B/s]Downloading (…)55de9125/config.json: 100%|██████████| 612/612 [00:00<00:00, 196kB/s]
Downloading (…)ce_transformers.json: 0%| | 0.00/116 [00:00<?, ?B/s]Downloading (…)ce_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 37.6kB/s]
Downloading (…)125/data_config.json: 0%| | 0.00/39.3k [00:00<?, ?B/s]Downloading (…)125/data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 3.09MB/s]
Downloading pytorch_model.bin: 0%| | 0.00/90.9M [00:00<?, ?B/s]Downloading pytorch_model.bin: 12%|█▏ | 10.5M/90.9M [00:00<00:01, 59.0MB/s]Downloading pytorch_model.bin: 46%|████▌ | 41.9M/90.9M [00:00<00:00, 151MB/s] Downloading pytorch_model.bin: 81%|████████ | 73.4M/90.9M [00:00<00:00, 205MB/s]Downloading pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:00<00:00, 190MB/s]
Downloading (…)nce_bert_config.json: 0%| | 0.00/53.0 [00:00<?, ?B/s]Downloading (…)nce_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 6.65kB/s]
Downloading (…)cial_tokens_map.json: 0%| | 0.00/112 [00:00<?, ?B/s]Downloading (…)cial_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 36.1kB/s]
Downloading (…)e9125/tokenizer.json: 0%| | 0.00/466k [00:00<?, ?B/s]Downloading (…)e9125/tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 7.52MB/s]
Downloading (…)okenizer_config.json: 0%| | 0.00/350 [00:00<?, ?B/s]Downloading (…)okenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 109kB/s]
Downloading (…)9125/train_script.py: 0%| | 0.00/13.2k [00:00<?, ?B/s]Downloading (…)9125/train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 4.05MB/s]
Downloading (…)7e55de9125/vocab.txt: 0%| | 0.00/232k [00:00<?, ?B/s]Downloading (…)7e55de9125/vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 6.28MB/s]
Downloading (…)5de9125/modules.json: 0%| | 0.00/349 [00:00<?, ?B/s]Downloading (…)5de9125/modules.json: 100%|██████████| 349/349 [00:00<00:00, 105kB/s]

In [ ]:
df_time = df_time.withColumnRenamed('Total time','total_time').withColumn('cooking_steps_emb',get_embeddings_udf("cooking_steps"))
df_time.show(1) #We can see the embedded directions in last column

+--------------------+----------+--------------------+--------------------+
 name|total_time| cooking_steps| cooking_steps_emb|
+--------------------+----------+--------------------+--------------------+
Irish Black Russi...| 5 mins|Halffill a highba...|[-0.005568775, -0...|
+--------------------+----------+--------------------+--------------------+
only showing top 1 row

<h4>We create a dense vector which has same dimensions and all values are non-zero

In [ ]:
#To remove the sparscity in the embeddings
def get_dense_vector(embeddings):
    return Vectors.dense(embeddings)
 
get_dense_vector_udf = udf(get_dense_vector, VectorUDT())

In [ ]:
df_time = df_time.filter(df_time.total_time.isNotNull()) #Filtering null values from our label(cooking time = 0 for few items)

<h4>We convert the label column - total_time from days, hours and minutes to minutes only

In [ ]:
def label_to_minutes(label):
    parts = label.split()
    total_minutes = 0
    for i in range(0, len(parts), 2):
        value = int(parts[i])
        unit = parts[i+1]
        if unit.startswith("hr"):
            total_minutes += value * 60
        elif unit.startswith("day"):
            total_minutes += value * 24 * 60
        elif unit.startswith("min"):
            total_minutes += value
    return int(total_minutes)

label_to_minutes_udf = udf(label_to_minutes)

In [ ]:
df_time_new = df_time.withColumn('cooking_steps_dense',get_dense_vector_udf('cooking_steps_emb'))\
                        .withColumn('label',label_to_minutes_udf('total_time'))\
                        .withColumn("label", col("label").cast("int"))
df_time_new.printSchema()

root
-- name: string (nullable = true)
-- total_time: string (nullable = true)
-- cooking_steps: string (nullable = true)
-- cooking_steps_emb: array (nullable = true)
 |-- element: float (containsNull = true)
-- cooking_steps_dense: vector (nullable = true)
-- label: integer (nullable = true)

<h4>We use VectorAssembler to get our features ready and fit in RandomForestRegressor.</h4>
We used 50 trees with 15 depth to fit the model (this was found using CV but removed due to processesing power)

In [ ]:
assembler = VectorAssembler(inputCols=["cooking_steps_dense"], outputCol="features")
df_vector = assembler.transform(df_time_new).select("features", "label")
featureSet = df_vector.randomSplit([0.8, 0.2], 1)
feature_train = featureSet[0]
feature_valid = featureSet[1]
feature_train = feature_train.repartition(50)
feature_valid = feature_valid.repartition(50)
feature_train.cache()
feature_train.cache() #We cache and repartition it make it more efficient

Out[16]: DataFrame[features: vector, label: int]

In [ ]:
feature_train.printSchema()

root
-- features: vector (nullable = true)
-- label: integer (nullable = true)

In [ ]:
# #NULL Check
# feature_train.filter(feature_train.features.isNull()).count(),feature_train.filter(feature_train.label.isNull()).count(),feature_valid.filter(feature_valid.features.isNull()).count(),feature_valid.filter(feature_valid.label.isNull()).count()

In [ ]:
rf = RandomForestRegressor(numTrees=50, maxDepth=15,seed=42)
rfmodel = rf.fit(feature_train)

In [ ]:
predictions = rfmodel.transform(feature_valid)
predictions.cache() #Cache for easier availability

Out[20]: DataFrame[features: vector, label: int, prediction: double]

In [ ]:
predictions.printSchema()

root
-- features: vector (nullable = true)
-- label: integer (nullable = true)
-- prediction: double (nullable = false)

In [ ]:
# create a RegressionEvaluator object with R2 as the evaluation metric
evaluator = RegressionEvaluator(metricName="r2").setLabelCol("label")\
                                                 .setPredictionCol("prediction")

r2 = evaluator.evaluate(predictions)

print("R-squared (R2) on validation data = %g" % r2)

R-squared (R2) on validation data = 0.762437

In [ ]:
spark.stop()